# Preprocessing, Feature Extraction and Modelling

In [5]:
import os
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from skimage import io, color, img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA

In [1]:
import os
from PIL import Image, ImageOps

root_dir = './data'

# Define the target size
target_size = (256, 256)

# Output directory for resized images
output_dir = './resize_data'

# Function to resize and add padding to an image
def resize_and_add_padding(image_path, output_path):
    image = Image.open(image_path)
    image = ImageOps.fit(image, target_size, method=0, bleed=0.0, centering=(0.5, 0.5))
    image.save(output_path)

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate through the "CORROSION" and "NOCORROSION" folders
for class_folder in ['CORROSION', 'NOCORROSION']:
    class_dir = os.path.join(root_dir, class_folder)

    # Create a subdirectory in the output directory for each class
    class_output_dir = os.path.join(output_dir, class_folder)
    os.makedirs(class_output_dir, exist_ok=True)

    # Iterate through the images in the class folder
    for filename in os.listdir(class_dir):
        if filename.endswith('.jpg'):  # Adjust the file extension as needed
            input_image_path = os.path.join(class_dir, filename)
            output_image_path = os.path.join(class_output_dir, filename)
            resize_and_add_padding(input_image_path, output_image_path)
            print(f"Resized: {input_image_path} -> {output_image_path}")


FileNotFoundError: [Errno 2] No such file or directory: './data/CORROSION'

## Feature Extraction

### Gray Level Co-Occurance Matrix

In [3]:
def compute_glcm_features(image_path):
    img = io.imread(image_path)
    gray = color.rgb2gray(img)
    image = img_as_ubyte(gray)
    
    bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255])  # 16-bit
    inds = np.digitize(image, bins)

    max_value = inds.max() + 1
    matrix_cooccurrence = graycomatrix(inds, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=max_value, normed=False, symmetric=False)

    contrast = graycoprops(matrix_cooccurrence, 'contrast')
    dissimilarity = graycoprops(matrix_cooccurrence, 'dissimilarity')
    homogeneity = graycoprops(matrix_cooccurrence, 'homogeneity')
    energy = graycoprops(matrix_cooccurrence, 'energy')
    correlation = graycoprops(matrix_cooccurrence, 'correlation')
    asm = graycoprops(matrix_cooccurrence, 'ASM')

    return {
        "Contrast": contrast,
        "Dissimilarity": dissimilarity,
        "Homogeneity": homogeneity,
        "Energy": energy,
        "Correlation": correlation,
        "ASM": asm
    }

In [4]:
corrosion_dir = "resize_data/CORROSION"
nocorrosion_dir = "resize_data/NOCORROSION"

# Function to compute GLCM features for a directory of images
def compute_glcm_features_for_directory(directory, label):
    features_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            image_path = os.path.join(directory, filename)
            features = compute_glcm_features(image_path)
            features["Label"] = label  # Add the label
            features_list.append(features)
    return features_list

# Compute GLCM features for both classes
corrosion_features = compute_glcm_features_for_directory(corrosion_dir, "corrosion")
nocorrosion_features = compute_glcm_features_for_directory(nocorrosion_dir, "nocorrosion")

# Combine features for both classes
all_features = corrosion_features + nocorrosion_features

# Save as csv file
feature_df = pd.DataFrame(all_features)
feature_df.to_csv("./features/glcm_features.csv", index=False)


In [4]:
import pandas as pd

# Read the original DataFrame
df_data = pd.read_csv('./features/glcm_features.csv')

# Extract the 'Contrast' column and remove square brackets
df_data['Contrast'] = df_data['Contrast'].str.replace('[', '').str.replace(']', '')
df_data['Dissimilarity'] = df_data['Dissimilarity'].str.replace('[', '').str.replace(']', '')
df_data['Homogeneity'] = df_data['Homogeneity'].str.replace('[', '').str.replace(']', '')
df_data['Energy'] = df_data['Energy'].str.replace('[', '').str.replace(']', '')
df_data['Correlation'] = df_data['Correlation'].str.replace('[', '').str.replace(']', '')
df_data['ASM'] = df_data['ASM'].str.replace('[', '').str.replace(']', '')



# Split the 'Contrast' column values by whitespace and expand them into separate columns
df_data[['Contrast0', 'Contrast45', 'Contrast90', 'Contrast135']] = df_data['Contrast'].str.split(expand=True)
df_data[['Dissimilarity0', 'Dissimilarity45', 'Dissimilarity90', 'Dissimilarity135']] = df_data['Dissimilarity'].str.split(expand=True)
df_data[['Homogeneity0', 'Homogeneity45', 'Homogeneity90', 'Homogeneity135']] = df_data['Homogeneity'].str.split(expand=True)
df_data[['Energy0', 'Energy45', 'Energy90', 'Energy135']] = df_data['Energy'].str.split(expand=True)
df_data[['Correlation0', 'Correlation45', 'Correlation90', 'Correlation135']] = df_data['Correlation'].str.split(expand=True)
df_data[['ASM0', 'ASM45', 'ASM90', 'ASM135']] = df_data['ASM'].str.split(expand=True)


# Drop the original 'Contrast' column
df_data.drop(columns=['Contrast'], inplace=True)
df_data.drop(columns=['Dissimilarity'], inplace=True)
df_data.drop(columns=['Homogeneity'], inplace=True)
df_data.drop(columns=['Energy'], inplace=True)
df_data.drop(columns=['Correlation'], inplace=True)
df_data.drop(columns=['ASM'], inplace=True)


# Save the new DataFrame to a new CSV file
new_csv_filename = './features/glcm_features_split.csv'
df_data.to_csv(new_csv_filename, index=False)

## Dengan PCA

In [19]:
data = pd.read_csv('./features/glcm_features_split.csv')


features = data.drop('Label', axis=1)

# Standardize the features
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# Encode labels (corrosion, no corrosion) to numerical values
label_encoder = LabelEncoder()
y = data['Label']
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_standardized, y, test_size=0.2, random_state=42)

# Initialize and train the MLP classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, solver='sgd', random_state=42, verbose=False)
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f"Classification Report:\n{classification_rep}")


Classification Report:
              precision    recall  f1-score   support

   corrosion       0.73      0.81      0.77       192
 nocorrosion       0.75      0.64      0.69       166

    accuracy                           0.73       358
   macro avg       0.74      0.73      0.73       358
weighted avg       0.74      0.73      0.73       358



/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Load your dataset (assuming 'glcm_features_new.csv')
data = pd.read_csv('./features/glcm_features_split.csv')

features = data.drop('Label', axis=1)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)


n_components = 15

for i in range(n_components):
    # Apply PCA
    pca = PCA(n_components=i+1)
    pca.fit(features_standardized)

    # Transform the features
    features_pca = pca.transform(features_standardized)

    # Encode labels (corrosion, no corrosion) to numerical values
    label_encoder = LabelEncoder()
    y = data['Label']
    y = label_encoder.fit_transform(y)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features_pca, y, test_size=0.2, random_state=42)

    # Initialize and train the MLP classifier
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(300, ), max_iter=200, random_state=42, verbose=False, solver='adam',alpha=0.0001,
                                activation='relu', learning_rate='constant')
    mlp_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = mlp_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

    # Print the results
    print(f"Accuracy: {accuracy}, pca_component = {i+1}")
print(f"Classification Report:\n{classification_rep}")


Accuracy: 0.6256983240223464, pca_component = 1
Accuracy: 0.6256983240223464, pca_component = 2
Accuracy: 0.6787709497206704, pca_component = 3
Accuracy: 0.7374301675977654, pca_component = 4


/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.7402234636871509, pca_component = 5


/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.770949720670391, pca_component = 6


...

/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.7821229050279329, pca_component = 13


/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.7849162011173184, pca_component = 14
Accuracy: 0.7988826815642458, pca_component = 15
Classification Report:
              precision    recall  f1-score   support

   corrosion       0.81      0.82      0.81       192
 nocorrosion       0.79      0.77      0.78       166

    accuracy                           0.80       358
   macro avg       0.80      0.80      0.80       358
weighted avg       0.80      0.80      0.80       358



/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Saving The Model

In [26]:
import joblib

# Save the trained MLP classifier to a file
model_filename = './model/mlp_classifier_model.joblib'
joblib.dump(mlp_classifier, model_filename)

['./model/mlp_classifier_model.joblib']

#### Main Code Prediction

In [36]:
import joblib
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import numpy as np

model_filename = './model/mlp_classifier_model.joblib'
loaded_mlp_classifier = joblib.load(model_filename)

# Load the trained MLP classifier model from a filebackup FEATURE/feature_extraction.ipynb
image_path = './resize_data/NOCORROSION/100e35cf19.jpg'
target_size = (256, 256)

image = Image.open(image_path)

# Resize Image
image_new = ImageOps.fit(image, target_size, method=0, bleed=0.0, centering=(0.5, 0.5))
temp_image_path = './temp_resized_image.jpg'
image_new.save(temp_image_path)

# Extract GLCM Value
features = compute_glcm_features(temp_image_path)

# Initialize a new dictionary to store the transformed features
transformed_features = {}

# Define a list of angles
angles = ['0', '45', '90', '135']

# Iterate through the features and angles to create new labels
for feature_name, feature_values in features.items():
    for i, angle in enumerate(angles):
        new_label = f'{feature_name}{angle}'
        transformed_features[new_label] = feature_values[0][i]

# Transform the dictionary to a 1D NumPy array
transformed_features_array = np.array(list(transformed_features.values())).reshape(1, -1)

# Use PCA to reduce the dimensionality to 15
pca = PCA(n_components=15)
transformed_features_pca = pca.fit_transform(transformed_features_array.reshape(1, -1))

# Make predictions using the loaded model
predictions = loaded_mlp_classifier.predict(transformed_features_pca)

if predictions[0] == 0:
    print("HASIL PREDIKSI: Corrosion")
else:
    print("HASIL PREDIKSI: NoCorrosion")

# Close and remove the temporary image file
image_new.close()
if os.path.exists(temp_image_path):
    os.remove(temp_image_path)

image = plt.imread(image_path)
plt.imshow(image)

ValueError: n_components=15 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

### Local Binary Pattern

In [38]:
def read_images_from_folder(folder_path):
    images = []
    labels = []

    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.isdir(subfolder_path):
            label = subfolder  # Use the subfolder name as the label
            for fn in os.listdir(subfolder_path):
                if fn.endswith('.jpg'):
                    img_path = os.path.join(subfolder_path, fn)
                    im = Image.open(img_path).convert('L')
                    data = np.array(im)
                    images.append(data)
                    labels.append(label)

    return images, labels

# Load images and labels from the 'resize_data' folder structure
data_folder = './resize_data'
images, labels = read_images_from_folder(data_folder)
print('Load data success!')

X = np.array(images)
print(X.shape)

# Encode labels (CORROSION, NOCORROSION) to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

radius = 2
n_point = radius * 8

Load data success!
(1790, 256, 256)
(1432, 256, 256)
(358, 256, 256)
(1432,)
(358,)


In [39]:
def lbp_texture(train_data, test_data):
    max_bins_train = 0
    max_bins_test = 0

    for i in range(len(train_data)):
        lbp = feature.local_binary_pattern(train_data[i], n_point, radius, 'default')
        max_bins_train = max(max_bins_train, int(lbp.max()) + 1)

    for i in range(len(test_data)):
        lbp = feature.local_binary_pattern(test_data[i], n_point, radius, 'default')
        max_bins_test = max(max_bins_test, int(lbp.max()) + 1)

    train_hist = np.zeros((len(train_data), max_bins_train))
    test_hist = np.zeros((len(test_data), max_bins_test))

    for i in range(len(train_data)):
        lbp = feature.local_binary_pattern(train_data[i], n_point, radius, 'default')
        train_hist[i], _ = np.histogram(lbp, bins=max_bins_train, density=True)

    for i in range(len(test_data)):
        lbp = feature.local_binary_pattern(test_data[i], n_point, radius, 'default')
        test_hist[i], _ = np.histogram(lbp, bins=max_bins_test, density=True)

    return train_hist, test_hist


In [40]:
from sklearn.neural_network import MLPClassifier
from skimage import feature
from PIL import Image
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X_train, X_test = lbp_texture(X_train, X_test)

# Create and train an MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=200)
mlp.fit(X_train, y_train)

# Predict on the test set
y_pred = mlp.predict(X_test)

# Evaluate the MLP classifier
train_accuracy = mlp.score(X_train, y_train)
test_accuracy = mlp.score(X_test, y_test)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
classify_report = classification_report(y_test, y_pred)


print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"\nOverall Accuracy: {classify_report}")

/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training Accuracy: 0.9636871508379888
Test Accuracy: 0.8491620111731844
Precision: 0.8502792586951002
Recall: 0.8463227911646587
F1 Score: 0.8476211495412556
Overall Accuracy: 0.8491620111731844
Overall Accuracy:               precision    recall  f1-score   support

           0       0.84      0.89      0.86       192
           1       0.86      0.81      0.83       166

    accuracy                           0.85       358
   macro avg       0.85      0.85      0.85       358
weighted avg       0.85      0.85      0.85       358



In [41]:
import joblib

# Save the trained MLP model to a file
model_filename = './model/mlp_lbp_model.pkl'
joblib.dump(mlp, model_filename)

print(f"MLP model saved as {model_filename}")

MLP model saved as ./model/mlp_lbp_model.pkl


In [34]:
from skimage import feature
from PIL import Image
import numpy as np

def lbp_texture(image):
    # Perform LBP feature extraction on a single image
    lbp = feature.local_binary_pattern(image, n_point, radius, 'default')
    max_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, bins=max_bins, density=True)
    return hist

# Load and preprocess a single image
image_path = './resize_data/NOCORROSION/01a5aee1ab.jpg'
im = Image.open(image_path).convert('L')
data = np.array(im)

# Define LBP parameters
radius = 2
n_point = radius * 8

# Call the LBP function on the single image
lbp_features = lbp_texture(data)

loaded_model = joblib.load('model/mlp_lbp_model.pkl')
prediction = loaded_model.predict([lbp_features])
print(prediction)


FileNotFoundError: [Errno 2] No such file or directory: 'model/mlp_lbp_model.pkl'

### Model Evaluation